# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD5 import SEIRHVD 
from Quarantine import Quarantine
from Quarantine import SeroPrevDynamics
from importdata import ImportData

Windows


## Time Variables
Declare Initial date for simulating and data fitting

In [45]:
# First simulation Initial date
initdate = datetime(2020,5,15)
# Date for change in SeroPrevalence
SPchange_date = datetime(2020,8,24)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. RM region is represented by cut = 13.

In [46]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data 2
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Importing Hospitalized/NonHospitalized Deaths
Done


## Datos de Hospitalizados a fitear

In [47]:
#RM.Hr = np.array(RM.UCI) + np.array(RM.UTI)
#RM.Hr_tot = np.array(RM.UCI_tot) + np.array(RM.UTI_tot)
RM.Hr = np.array(RM.UTI)
RM.Hr_tot = np.array(RM.UTI_tot)

## Simulation Parameters

In [774]:
# Total simulation time
tsim = 1000

t_sp = (SPchange_date - initdate).days 


beta = 0.12
mu = 1.6
k_I = 0
k_R = 0

SeroPrevFactor =0.07

expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [820]:
# Quarantines 
max_mob = 0.6
rem_mob = 0.45

Q1 = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=t_sp)
Q2 = Quarantine(rem_mob)

alpha = Q1.alpha

## Underreport
Fraction of Infected detected/reported  

$Imi_{det}$: Fraction of Mild detected   
$Ias_{det}$: fraction of asymptomatic detected  

If both are 1, means that all infected are detected so 
\begin{align}
I_{det} = I
\end{align}

In [821]:
Imi_det = 1
Ias_det = 0.15
Ise_det = 1
Icr_det = 1

## State Parameters:
The following parameters determine transitions probability and duration between different states. The values presented here are the default values set in the model. You can change them as it's showed in the following executable lines.

pE_Ias = 0.4  # Transition from exposed to Asymptomatic Infected  
tE_Ias = 5.0

pE_Imi = 0.55 # Transition from exposed to  Mild Infected  
tE_Imi = 5.0

pE_Icr = 0.01666 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.03334 # Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 10.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 15.0

pIse_Hse = 1.0 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 3.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered  
tHse_R = 11.0

pHse_V = 0.03 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 3.0

pHse_D = 0.03 # Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 3.0        

pV_Hout = 0.5  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 15.0

pV_D = 0.5 # Transition from Ventilators to Death  
tV_D = 15.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 4.0

pD_B = 1.0 # Transition from Dead to buried  
tD_B = 1.0 

betaD = 0 # Contagion by deads rate  
eta = 0.0 # Immunity loss rate  

In [909]:
pE_Ias = 0.428 # Transition from exposed to Asymptomatic Infected  
tE_Ias = 4.0

pE_Imi = 0.528 # Transition from exposed to  Mild Infected  
tE_Imi = 4.0

pE_Icr = 0.012 # Transition from exposed to  Critical Infected  
tE_Icr = 3.0

pE_Ise = 0.032# Transition from exposed to  Serious Infected  
tE_Ise = 3.0

pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered  
tIas_R = 9.0

pImi_R = 1.0  # Transition from Mild Infected to Recovered  
tImi_R = 12.0

pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)  
tIse_Hse = 5.0 

pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)  
tIse_D = 3.0         

pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)  
tIcr_V = 3.0 

pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)  
tIcr_D = 3.0         

pHse_R = 0.95 # Transition from Serious Hospitalized to Recovered  
tHse_R = 3.0

pHse_V = 0.05 # Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)  
tHse_V = 5.0

pHse_D = 1# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)  
tHse_D = 5.0        

pV_Hout = 0.4  # Transition from Ventilators to Hospital Recovery (Hout)   
tV_Hout = 18.0

pV_D = 0.6 # Transition from Ventilators to Death  
tV_D = 12.0

pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered  
tHout_R = 20.0

## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

In [945]:
# Days for susceptible increase
renewalFactor=0.6
initincrease=0
increasedays =initincrease+30
# Daily amount of people 
dailyincrease = RM.population*SeroPrevFactor*renewalFactor/increasedays

In this case we add 30.000 persons to the simulation

In [946]:
chi = SeroPrevDynamics(t_sp-initincrease,t_sp-initincrease+increasedays,dailyincrease)

We can see the SeroPrev addition in the following plot

In [947]:
plt.plot(range(200),[chi(i) for i in range(200)])

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

In [949]:
simulation1 = SEIRHVD(tsim,beta,mu,alpha,k_I=k_I,k_R = k_R, chi = chi, SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,RealIC = RM,Imi_det = Imi_det,Ias_det = Ias_det)

InitialCondition Object Data


Modify simulation state parameters

In [950]:
simulation1.pE_Ias=pE_Ias
simulation1.tE_Ias=tE_Ias
simulation1.pE_Imi=pE_Imi
simulation1.tE_Imi=tE_Imi
simulation1.pE_Icr=pE_Icr
simulation1.tE_Icr=tE_Icr
simulation1.pE_Ise=pE_Ise
simulation1.tE_Ise=tE_Ise
simulation1.pIas_R=pIas_R
simulation1.tIas_R =tIas_R
simulation1.pImi_R=pImi_R
simulation1.tImi_R =tImi_R
simulation1.pIse_Hse=pIse_Hse
simulation1.tIse_Hse=tIse_Hse
simulation1.pIse_D=pIse_D
simulation1.tIse_D=tIse_D
simulation1.pIcr_V=pIcr_V
simulation1.tIcr_V=tIcr_V
simulation1.pIcr_D=pIcr_D
simulation1.tIcr_D=tIcr_D
simulation1.pHse_R=pHse_R
simulation1.tHse_R =tHse_R
simulation1.pHse_V=pHse_V
simulation1.tHse_V=tHse_V
simulation1.pHse_D=pHse_D
simulation1.tHse_D=tHse_D
simulation1.pV_Hout=pV_Hout
simulation1.tV_Hout =tV_Hout
simulation1.pV_D=pV_D
simulation1.tV_D =tV_D
simulation1.pHout_R=pHout_R
simulation1.tHout_R=tHout_R
simulation1.setnewparams()

Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [951]:
simulation1.integr_sci(0,tsim,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 4425
     njev: 81
      nlu: 81
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.55007863e-08, 3.10015727e-08, 1.55038865e-04,
       3.10046728e-04, 4.65054591e-04, 2.01513322e-03, 3.56521186e-03,
       5.11529049e-03, 6.66536912e-03, 2.21661555e-02, 3.76669418e-02,
       5.31677281e-02, 6.86685144e-02, 8.41693008e-02, 2.39177164e-01,
       3.94185027e-01, 5.49192891e-01, 7.04200754e-01, 8.59208617e-01,
       1.01421648e+00, 1.42876084e+00, 1.84330520e+00, 2.25784957e+00,
       2.67239393e+00, 3.08693829e+00, 3.50148265e+00, 3.91602701e+00,
       4.33057137e+00, 4.87778744e+00, 5.42500350e+00, 5.97221956e+00,
       6.51943563e+00, 7.04493734e+00, 7.57043906e+00, 8.09594077e+00,
       8.97847850e+00, 9.86101623e+00, 1.07435540e+01, 1.18111388e+01,
       1.28787236e+01, 1.39463084e+01, 1.54041908e+01, 1.68620731e+01,
       1.83199554e+01, 1.90338488e+01

# Analysis

## Grid Plot

In [952]:
# Days to plot
days = 250
t_end = t_sp

In [953]:
fig, axs = plt.subplots(2, 3)
axs[0,0].set_xlim(0,days)
axs[0,0].scatter(RM.tr,RM.Ir,label='Real Active Data')
axs[0,0].plot(simulation1.t,simulation1.I_det,label='Infected')
axs[0,0].set_title('Active Infected')
axs[0,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,1].set_xlim(0,days)
axs[0,1].plot(simulation1.t,simulation1.I_d_det,label='sim1')
axs[0,1].scatter(RM.I_d_r_tr,RM.I_d_r,label='Daily Real Data')
axs[0,1].set_title('Daily infected')
axs[0,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,2].set_xlim(0,days)
axs[0,2].plot(simulation1.t,simulation1.I_ac_det,label='sim1')
axs[0,2].scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Accumulated Real Data')
axs[0,2].set_title('Accumulated Infected')
axs[0,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,0].set_xlim(0,days)
axs[1,0].plot(simulation1.t,simulation1.B,label='sim1')
axs[1,0].scatter(RM.hosp_tr,RM.Br_hosp,label='Real Data')
axs[1,0].set_title('Hospitalized Acmulated Deaths')
axs[1,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,1].set_xlim(0,days)
axs[1,1].plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UTI Beds')
axs[1,1].scatter(RM.sochimi_tr,RM.UCI,label='Real Data')
axs[1,1].plot(simulation1.t,simulation1.H_cap,label='Capacity', color = 'red')
axs[1,1].set_title('UTI Usage')
axs[1,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,2].set_xlim(0,days)
axs[1,2].plot(simulation1.t,simulation1.V,label='Vsat sim1')
axs[1,2].scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
axs[1,2].plot(simulation1.t,simulation1.V_cap,label='Capacity Data', color = 'red')
axs[1,2].set_title('VMI Usage')
axs[1,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')



## Single plots

### SEIRD

In [926]:
plt.plot(simulation1.t,simulation1.S,label='S')
plt.plot(simulation1.t,simulation1.E,label='E')
plt.plot(simulation1.t,simulation1.I,label='I')
plt.plot(simulation1.t,simulation1.R,label='R')
plt.plot(simulation1.t,simulation1.B,label='B')
plt.axvline(x = t_end, linestyle = 'dotted',color = 'grey')
plt.title('SEIRD')
plt.legend(loc=0)
plt.show()

### Daily Deaths

In [841]:
plt.plot(simulation1.t,simulation1.D,label='sim1')
plt.title('Deaths')
plt.show()

### VMI Usage

In [842]:
plt.plot(simulation1.t,simulation1.V_cap,label='VMI capacity sim1')
plt.plot(simulation1.t,simulation1.V,label='VMI sim1')
plt.legend(loc=0)
plt.title('VMI')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Vr_tot,label='Real Data')
plt.show()

### Hospitalization

In [843]:
plt.plot(simulation1.t,simulation1.H_cap,label='sim1')
plt.plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UCI/UTI Beds')
plt.legend(loc=0)
plt.title('Hospitalization')

plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.show()

### Deaths by cause

In [ ]:
plt.plot(simulation1.t,simulation1.D,label='sim1')
plt.plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
plt.plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
plt.plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
plt.plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')

plt.legend(loc=0)
plt.title('Deaths by cause')
plt.show()

### Hospital saturation

In [ ]:
plt.plot(simulation1.t,simulation1.H_sat,label='HSat')
plt.plot(simulation1.t,simulation1.V_sat,label='Vsat')
plt.legend(loc=0)
plt.title('Saturation')
plt.show()

# Error Calculation
The following plots will show the simulation results with the real data

## Peak Date

In [ ]:
peakidx = np.where(RM.Ir==max(RM.Ir))[0][0]
datapeakdate = RM.sochimi_dates[peakidx]

In [ ]:
simulation.peak_date

In [ ]:
datapeakdate

In [ ]:
Peak_delta = (simulation.peak_date -datapeakdate).days

In [ ]:
Peak_delta

## Errors

In [ ]:
simulation = simulation1
t_end = tsim1
t_end_idx = np.where(np.array(RM.tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t_end_idx])
E_I = np.sum(abs(RM.Ir[:t_end_idx]-simulation.I_det[idx]))/(np.mean(RM.Ir[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_d_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_d_r_tr[:t_end_idx])
E_Id = np.sum(abs(RM.I_d_r[:t_end_idx]-simulation.I_d_det[idx]))/(np.mean(RM.I_d_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_ac_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_ac_r_tr[:t_end_idx])
E_Iac = np.sum(abs(RM.I_ac_r[:t_end_idx]-simulation.I_ac_det[idx]))/(np.mean(RM.I_ac_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.Br_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.Br_tr[:t_end_idx])
E_D = np.sum(abs(RM.Br[:t_end_idx]-simulation.B[idx]))/(np.mean(RM.Br[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_UCI = np.sum(abs(RM.Hr[:t_end_idx]-(simulation.Hse[idx]+simulation.Hout[idx])))/(np.mean(RM.Hr[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_VMI = np.sum(abs(RM.Vr[:t_end_idx]-simulation.V[idx]))/(np.mean(RM.Vr[:t_end_idx])*t_end_idx)

Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}


In [ ]:
Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}
print(Err)